In [113]:
import numpy as np
from sympy import Matrix # Used to compute nullity

In [114]:
G = 'B' # Type of classical algebraic group, Input: string 'A','B','C', or 'D'
l = 2 # Rank, Input: positive integer
n = 2*l+1 # Square matrix dimension for type B

In [115]:
"""
Replace all the .replace("(", "").replace(")", "").replace(" ", "").replace(",", "") with this
"""
def tup_to_str(tuple):
    return str(tuple).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")

In [116]:
"""
Make matrices for the root vectors for the positive roots in type B (i.e., the root vectors in the nilradical).
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays

"mat_type" is the eigenvector matrix form given on page 131 of Erdmann
"indices" is a list of the i,j indices
"matrix" is the eigenvector corresponding to the root of specified type and indices
"coords" are the coordinates of the positive nonzero matrix entry, to be used for the coordinate function

Parameters: rank l

Returns: dictionary

"""
def make_rvecs_B(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1),dtype = int)
    # Add root vectors b_i for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        M[i,0] = 1
        M[0,l+i] = -1
        hilo = "b" + str(i)
        coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
        coef[i-1:l] = 1
        coef = tuple(coef)
        rvecs.setdefault(coef, {"matrix":M, "coords":[[i,0],[0,l+i]]}) # index 1 \leq i \leq l
        M = np.zeros((2*l+1,2*l+1),dtype = int)
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j] = 1
                coef[j-1:l] = 2
                coef = tuple(coef)
                rvecs.setdefault(coef, {"matrix":M, "coords":[[i,l+j],[j,l+i]]}) # indices 1 \leq i < j \leq l
                M = np.zeros((2*l+1,2*l+1),dtype = int)
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j-1] = 1 
                coef = tuple(coef)
                rvecs.setdefault(coef, {"matrix":M, "coords":[[i,j],[l+j,l+i]]})
                M = np.zeros((2*l+1,2*l+1), dtype = int)
    return rvecs

rvecs = make_rvecs_B(l)
print(rvecs)

{(1, 1): {'matrix': array([[ 0,  0,  0, -1,  0],
       [ 1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]]), 'coords': [[1, 0], [0, 3]]}, (0, 1): {'matrix': array([[ 0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]]), 'coords': [[2, 0], [0, 4]]}, (1, 2): {'matrix': array([[ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]]), 'coords': [[1, 4], [2, 3]]}, (1, 0): {'matrix': array([[ 0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  0]]), 'coords': [[1, 2], [4, 3]]}}


In [117]:
"""
Fix a sequence of the positive roots with a (partial) ordering induced by increasing root height.

Use the Sage function positive_roots_by_height(increasing=True)

Parameters: Group type G, string
            Rank, integer

Returns: List of roots as tuples
"""
def make_root_order(G,l):
    L = RootSystem([G,l]).root_lattice()
    roots = [tuple(root.to_vector()) for root in L.positive_roots_by_height()]
    return roots

roots = make_root_order(G,l)
num_roots = len(roots) # Store the number of positive roots for future loops

In [118]:
"""
VERSION WHERE THE ENTRIES ARE THE DIFFERENT THAN THOSE IN THE NILRADICAL (r INSTEAD OF n)
Compute the root spaces $\mathfrak{g}_\beta=\mathbb{C}e_\beta$

Notes:
This could be combined with the function that makes the nilradical
The "DeprecationWarning: invalid escape sequence \m" that returns when this cell is run in Jupyter is because of the 
backslash character used in the TeX for this comment

Parameters: rvecs

Returns: rspaces: dictionary with keys the roots as tuples and values the root spaces
"""
def make_root_spaces(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("r"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    return rspaces

rspaces = make_root_spaces(rvecs)

<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<ipython-input-118-969feb4a3bad>:13: DeprecationWarning: invalid escape sequence \m
  """


In [119]:
"""
VERSION WHERE THE ENTRIES ARE THE SAME AS IN THE NILRADICAL
Compute the root spaces $\mathfrak{g}_\beta=\mathbb{C}e_\beta$

Notes:
This could be combined with the function that makes the nilradical
The "DeprecationWarning: invalid escape sequence \m" that returns when this cell is run in Jupyter is because of the 
backslash character used in the TeX for this comment

Parameters: rvecs

Returns: rspaces: dictionary with keys the roots as tuples and values the root spaces
"""
def make_root_spaces(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("n"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    return rspaces

rspaces = make_root_spaces(rvecs)

<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<ipython-input-119-6312676660ef>:13: DeprecationWarning: invalid escape sequence \m
  """


In [120]:
rspaces

{(1,
  1): array([[0, 0, 0, -n11, 0],
        [n11, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=object),
 (0,
  1): array([[0, 0, 0, 0, -n01],
        [0, 0, 0, 0, 0],
        [n01, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=object),
 (1,
  2): array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, n12],
        [0, 0, 0, -n12, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=object),
 (1,
  0): array([[0, 0, 0, 0, 0],
        [0, 0, n10, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, -n10, 0]], dtype=object)}

In [121]:
"""
Compute the coordinates of the not-necessarily-zero matrix entries associated to the root spaces

Params: rvecs
Returns: rspace_coords: Dictionary containing root tuples as keys and nonzero (not necessarily zero) coordinates as values
"""

def make_rspace_coords(rvecs):
    # Dictionary containing root tuples as keys and nonzero (not necessarily zero) coordinates as values
    rspace_coords = {list(rvecs.keys())[i]:list(rvecs.values())[i]['coords'] for i in range(len(rvecs))}
    return rspace_coords
rspace_coords = make_rspace_coords(rvecs)

In [122]:
"""
Compute a matrix representing an arbitrary element of the nilradical

Input: rvecs
Returns: numpy.ndarray ; Arbitrary element of the nilradical
"""
def make_nilrad(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("n"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    nil_rad = sum(mats) # Set an arbitrary element of the nilradical equal to the sum of the root spaces
    
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    
    return nil_rad #,rspaces, mats

nil_rad = make_nilrad(rvecs)
nil_rad

array([[0, 0, 0, -n11, -n01],
       [n11, 0, n10, 0, n12],
       [n01, 0, 0, -n12, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, -n10, 0]], dtype=object)

In [123]:
"""
Compute the NT-submodules $\mathfrak{m}_i$
"""
def make_NT_submodules(n,roots,rspaces):
    mi_dict = {'m'+str(len(roots)): np.zeros((n,n), dtype = int)}
    for i in range(len(roots)):
        hilo = "m" + str(i)
        mi = rspaces[roots[i]]
        for k in range(i+1, len(roots)):
            mi += rspaces[roots[k]]
            #print(k,roots[k])

        mi_dict.setdefault(hilo,mi) #nil_rad-mi
    return mi_dict

mi_dict = make_NT_submodules(n,roots,rspaces)
mi_dict

<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<ipython-input-123-94be777e6d89>:3: DeprecationWarning: invalid escape sequence \m
  """


{'m4': array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]),
 'm0': array([[0, 0, 0, -n11, -n01],
        [n11, 0, n10, 0, n12],
        [n01, 0, 0, -n12, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, -n10, 0]], dtype=object),
 'm1': array([[0, 0, 0, -n11, 0],
        [n11, 0, n10, 0, n12],
        [0, 0, 0, -n12, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, -n10, 0]], dtype=object),
 'm2': array([[0, 0, 0, -n11, 0],
        [n11, 0, 0, 0, n12],
        [0, 0, 0, -n12, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=object),
 'm3': array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, n12],
        [0, 0, 0, -n12, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=object)}

In [124]:
"""
Compute the NT-submodules $\mathfrak{m}_i$ in terms of their not-necessarily-zero-coordinates using rspace_coords
"""
# Dictionary containing root tuples as keys and nonzero (not necessarily zero) coordinates as values
rspace_coords = {list(rvecs.keys())[i]:list(rvecs.values())[i]['coords'] for i in range(len(rvecs))}
def make_NT_submodule_coords(n,roots,rspaces):
    mi_dict_coords = {'m'+str(len(roots)): []}
    for i in range(len(roots)):
        hilo = "m" + str(i)
        mi = rspace_coords[roots[i]]
        for k in range(i+1, len(roots)):
            mi += rspace_coords[roots[k]]
            #print(k,roots[k])

        mi_dict_coords.setdefault(hilo,mi) #nil_rad-mi
    return mi_dict_coords

mi_dict_coords = make_NT_submodule_coords(n,roots,rspaces)
mi_dict_coords

<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<ipython-input-124-7264d48cac76>:3: DeprecationWarning: invalid escape sequence \m
  """


{'m4': [],
 'm0': [[2, 0], [0, 4], [1, 2], [4, 3], [1, 0], [0, 3], [1, 4], [2, 3]],
 'm1': [[1, 2], [4, 3], [1, 0], [0, 3], [1, 4], [2, 3]],
 'm2': [[1, 0], [0, 3], [1, 4], [2, 3]],
 'm3': [[1, 4], [2, 3]]}

In [125]:
"""
Function to compute the alpha-string through beta for the bracket [x_alpha,x_beta] (given by q and r, see Humphreys p. 145)

Parameters: alpha: tuple 
    beta: tuple
    roots: list of roots as tuples
    
Returns: q,r : integers

"""
def get_rstring(alpha, beta, roots):
    #rstring_dict = {}
    gamma = beta
    q = 0
    #print(gamma,type(gamma))
    while gamma in roots:
        gamma = tuple(np.add(gamma, alpha))
        if gamma in roots:
            q += 1
                
    gamma = beta
    r = 0
    while gamma in roots:
        gamma = tuple(np.subtract(gamma, alpha))
        if gamma in roots:
            r += 1
    return q,r

In [126]:
def Chev_bracket(alpha, beta, roots, rvecs):
    if alpha == beta:
        bracket = 0
    else:
        q, r = get_rstring(alpha,beta,roots)
        if q == 0:
            bracket = 0
        else:
            #print(rvecs[tuple(np.add(alpha,beta))]['matrix'])
            bracket = (r+1)*rvecs[tuple(np.add(alpha,beta))]['matrix'] #CHECK FOR SIGN CONVENTION
    return bracket

In [127]:
"""
Function to compute the dimension of the centralizer of x in u using the Chevalley commutator relations using the root vectors that were defined previously

define nil_coefs and vec_coefs as ORDERED DICTIONARIES where the keys are the roots and the values are the coefficients

Params: nil_coefs: list with the same order as roots of symbolic coefficient names nalpha1tuple etc. # ORDERED DICTIONARY where the keys are the roots and the values are the coefficients
    vec_coefs: ORDERED DICTIONARY where the keys are the roots and the values are the coefficients
    roots
    rvecs
    n (matrix dimension)

Returns: nullity
"""
def Chev_simp(nil_coefs, vec_coefs, roots, rvecs, n):
    mat = np.zeros((n,n),dtype = int)
    for alpha in roots: #nil_coefs: # nil_coefs.keys():
        for beta in vec_coefs: # .keys():
            bracket = Chev_bracket(alpha, beta, roots, rvecs)
            a = nil_coefs[alpha]
            b = vec_coefs[beta]
            mat = np.add(mat, a*b*bracket)
            
    return mat

In [128]:
"""
Params: mat: matrix (numpy or probably some other kinds work too)

Returns: nullity: integer
"""

def nullity(mat):
    # Using Sympy      
    mat = Matrix(mat)
    NumCol = mat.shape[1]
    rank = mat.rank()
    # Nullity of the matrix
    nullity = NumCol - rank
    
    return nullity

In [129]:
{key: value for key, value in X.items() if value != 0}

{}

In [133]:
def check_ramification(X, dimCu, nil_coefs, rspace_coords):
    # Remove root vectors with coefficient zero from X
    #vec_roots = {key: value for key, value in X.items() if value != 0}
    #vec_coefs = vec_roots.values()
    vec_coefs = {key: value for key, value in X.items() if value != 0}
    mat = Chev_simp(nil_coefs, vec_coefs, roots, rvecs, n)
    #print("pre",mat)
    # mod out by mi
    for j in range(len(mi)):
        mat[mi[j]] = 0
    #print("post",mat)
    if nullity(mat) == dimCu:
        #print(str(i) + " is a ramification point")
        ramification = True
    elif nullity(mat) == dimCu - 1:
        #print(str(i) + " is an inert point")
        dimCu = nullity(mat)
        ramification  = False
    else:
        print("something went wrong")
        ramification = None
        dimCu = nullity(mat)
        print(mat)
    return ramification, dimCu

In [178]:
for key in big_dict["level"+str(0)]:
    print(key)

0


In [198]:
"""
big_dict is a series of nested dictionaries
key: "level" + str(i)
value: dictionary with key X as a string and value a dictionary with key "dimCu" and corresponding value and 
"X_dict," a dictionary where the keys are root tuples and the values are their coefficients
Try printing big_dict["level0"] for a simple example
"""
nil_coefs = dict(zip(roots,[var("n"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(len(roots))]))


X = dict(zip(roots, np.zeros(n, dtype = int))) # Initial X: Dictionary where the keys are root tuples and the values are zero
k = ["level"+str(i) for i in range(len(roots)+1)]
v = [{} for i in range(len(roots)+1)]
big_dict = dict(zip(["level"+str(i) for i in range(len(roots)+1)],[{} for i in range(len(roots)+1)])) # Level number corresponds to mi--i.e., it is one greater than the index i used in the loop below
big_dict["level0"] = {"0":{"dimCu":n,"X_dict":dict(zip(roots, np.zeros(n, dtype = int)))}}

def min_reps(X,big_dict,roots):
    for i in range(1,len(roots)): # mi naming is one greater than the index i #for key in big_dict: 
        mi = mi_dict_coords["m" + str(i+1)]
        for key in big_dict["level"+str(i-1)]: # loop through the X as strings
            x = big_dict["level"+str(i)][key]["X_dict"]
            dimCu_prev = big_dict["level"+str(i)][key]["dimCu"]
            ramification, dimCu = check_ramification(x, dimCu_prev, nil_coefs, mi)
            if ramification == True:
                big_dict["level"+str(i+1)][key] = {}
                #print(big_dict["level"+str(i+1)][key])
                big_dict["level"+str(i+1)][key]["dimCu"] = dimCu
                big_dict["level"+str(i+1)][key]["X_dict"] = x
                
                
                big_dict["level"+str(i+1)]["keb" + str(i)] = {}
                # figure out how to do this better
                oop, dimCu_new = check_ramification(X, dimCu, nil_coefs, rspace_coords)
                big_dict["level"+str(i+1)]["keb" + str(i)]["dimCu"] = dimCu_new
                # and figure out how to do this better
                X_make = X
                hilo = var("k" + str(tup_to_str(roots[i-1])))
                X_make[roots[i]] = hilo
                #print(i,key,X_make)
                big_dict["level"+str(i+1)]["keb" + str(i)]["X_dict"] = X_make
                print("i",i,"key","X",key,big_dict["level"+str(i+1)]["keb" + str(i)]["X_dict"])
        #else:
            
    return big_dict
        
ohmy = min_reps(X,big_dict,roots)

KeyError: '0'

In [194]:
ohmy

{'level0': {'0': {'dimCu': 5,
   'X_dict': {(0, 1): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0}}},
 'level1': {},
 'level2': {},
 'level3': {},
 'level4': {}}

In [187]:
for key in ohmy["level1"]:
    print(ohmy["level1"][key])

{'dimCu': 5, 'X_dict': {(0, 1): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0}}
{'dimCu': 5, 'X_dict': {(0, 1): k01, (1, 0): k10, (1, 1): k11, (1, 2): k12}}


In [ ]:
"""
Most updated orbit representative "function"
"""

X = dict(zip(roots, np.zeros(n, dtype = int))) # Initial X: Dictionary where the keys are root tuples and the values are zero
dimCu = n
nil_coefs = dict(zip(roots,[var("n"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(len(roots))]))
#level_list = [X]
Xi_dict = {"level0":[X]}
for i in range(0,len(roots)):
    for x in range(len(Xi_dict["level"+str(i)])):
        mi = mi_dict_coords["m" + str(i)]
        # MAKE dimCu ASSOCIATED WITH X AND i--I DON'T THINK IT IS NOW
        ramification, dimCu = check_ramification(X, dimCu, nil_coefs, mi)
        if ramification == True:
            #Xkey = {key: value for key, value in X.items() if value != 0}.keys()
            Xi_new = X
            #print(roots[x])
            Xi_new[roots[i]] = var("k"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", ""))
            print("Xi_new",Xi_new)
            print(i)
            name = str(tuple(Xi_new.values())).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")
            print("name",name)
            # This line doesn't work--it puts exactly two representatives in each row
            Xi_dict.setdefault("level"+str(i+1)+Xi_new.values(), list(Xi_dict["level"+str(i)]) + [X, Xi_new])
        else:
            Xi_dict.setdefault("level"+str(i+1), list(Xi_dict["level"+str(i)]))
Xi_dict

In [ ]:
"""
Most updated but-under-construction orbit representative "function"
"""

X = dict(zip(roots, np.zeros(n, dtype = int))) # Initial X: Dictionary where the keys are root tuples and the values are zero
dimCu = n
nil_coefs = dict(zip(roots,[var("n"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(len(roots))]))
#level_list = [X]
Xi_dict = {"level0":[X]}
for i in range(0,len(roots)):
    for x in range(len(Xi_dict["level"+str(i)])):
        ramification, dimCu = check_ramification

In [ ]:
nil_coefs

In [ ]:
X = dict(zip(roots, np.zeros(n, dtype = int))) # Initial X: Dictionary where the keys are root tuples and the values are zero
dimCu = n
nil_coefs = [var("n"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(len(roots))]
Xi_dict = {} #{"level"+str(i) : X for i in range(1,len(roots)+1)}
#level_list = []
for i in range(1,len(roots)+1):
    for x in level_dict["level"+str(i)]:
        mi = mi_dict_coords["m"+str(i)]
        ramification, dimCu = check_ramification(X, dimCu, nil_coefs, mi)
        if ramification == True:
            Xkey = {key: value for key, value in X.items() if value != 0}.keys()
            Xi_dict.setdefault(str(Xkey)+str(i), rspaces[i+1])
    
#if ramification 

In [ ]:
k = var('k')
Chev_simp({(0,1):k,(1,0):1,(1,1):1,(1,2):1},{(0,1):1,(1,0):0,(1,1):0,(1,2):0},roots,rvecs,n)

In [ ]:
nil_rad

In [ ]:
"""
Calculation of U-orbits on u

"""
def make_row(previous_row_list, mi_dict):
    for i in range(len(roots)):
        mat = nil_rad - mi_dict['m'+str(i)]
        print(mat)
    

In [ ]:
X = 0


In [ ]:
for i in range(1,len(roots)):
    print(i)

In [ ]:
nil_coefs = [var("n"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(len(roots))]

In [ ]:
X.values()

In [ ]:
np.array(var("n"+str(roots[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")))

In [ ]:
rspaces

In [ ]:
alpha = (0,1,1)
beta = (0,0,1)
brute = rvecs[tuple(alpha)]['matrix'] @ rvecs[tuple(beta)]['matrix'] - rvecs[tuple(beta)]['matrix'] @ rvecs[tuple(alpha)]['matrix']
test = Chev_bracket((0,1,1),(0,0,1),roots, rvecs)
print(brute)
print(test)

In [ ]:
"""
Function to compute the alpha-string through beta (given by q and r, see Humphreys p. 145)
(to be used in order to simplify a matrix using the Chevalley 
commutator relations using the root vectors that were defined previously)

define nil_coefs and vec_coefs as ORDERED DICTIONARIES where the keys are the roots and the values are the coefficients
"""

def get_rstring_old(nil_coefs, vec_coefs, roots):
    rstring_dict = {}
    for alpha in range(len(list(nil_coefs.keys()))):
        for beta in range(len(list(vec_coefs.keys()))):
            # Compute the alpha-string through beta (alpha i-string through alpha j)
            gamma = list(vec_coefs.keys())[beta]
            q = 0
            while gamma in roots:
                gamma = tuple(np.add(gamma, list(nil_coefs.keys())[alpha]))
                #print(str(list(nil_coefs.keys())[alpha])+str(list(vec_coefs.keys())[beta]),gamma)
                if gamma in roots:
                    q += 1
                
            gamma = list(vec_coefs.keys())[beta]
            r = 0
            while gamma in roots:
                gamma = tuple(np.subtract(gamma, list(nil_coefs.keys())[alpha]))
                if gamma in roots:
                    r += 1
            rstring_dict.setdefault(str(list(nil_coefs.keys())[alpha])+str(list(vec_coefs.keys())[beta]),{'q':q,'r':r})
            # Compute the bracket
            #XiXj = 
        return rstring_dict

In [ ]:
"""
Function to simplify a matrix using the Chevalley commutator relations using the root vectors that were defined previously

define nil_coefs and vec_coefs as DICTIONARIES
"""

In [ ]:
"""
SCRATCH

Compute the NT-submodules $\mathfrak{m}_i$ as coordinates rather than matrices
"""
mi_dict = {}
for i in range(num_roots):
    hilo = "m" + str(i)
    mi = []
    mi.append(rvecs[roots[i]]['coords'])
    for k in range(i+1, num_roots):
        mi.append(rvecs[roots[k]]['coords'])
        #print(k,roots[k])
    
    mi_dict.setdefault(hilo,mi)
mi_dict

# Scratch

In [ ]:
# SCRATCH
m_list = []
for level in range(num_roots):
    mat = nil_rad
    #print(mat)
    for rspace in range(len(mi_dict['m'+str(level)])):
        print(mi_dict['m'+str(level)])
        coords = mi_dict['m'+str(level)][rspace]
        print(coords)
        mat[coords[0],coords[1]] = 0
        m_list.append(mat)

In [ ]:
cand = nil_rad - mi_dict['m0']

for i in range(len(roots)):
    cand = nil_rad - mi_dict['m' + str(i)]
    # Check if i is an inert point or a ramification point
    bracket = nil_rad @ cand - cand @ nil_rad
    print(bracket)

In [ ]:
list(rvecs.keys())[1]

In [ ]:
mats[1]

In [ ]:
(mats[0]+mats[1])@nil_rad-nil_rad@(mats[0]+mats[1])

In [ ]:
# Initialize vec_coefs as a numpy array with the greatest number of entries we could possibly need
# vec_coefs = np.zeros(len(roots),dtype = int)
# REWRITE THIS NOT TO USE APPEND
vec_roots = []
for key in X:
    if X[key] != 0:
        vec_roots.append(key)

vec_coefs

In [ ]:
nil_rad@rvecs[list(rvecs.keys())[1]]['matrix']-rvecs[list(rvecs.keys())[1]]['matrix']@nil_rad

In [ ]:
g = LieAlgebra(QQ, cartan_type=[G,l]) #, representation="matrix"

In [ ]:
B3 = RootSystem(["B",3]).root_space()
roots=list(B3.positive_roots())
roots

In [ ]:
g

In [ ]:
g.lie_algebra_generators()

In [ ]:
g.inject_variables()

In [ ]:
print(e1)

In [ ]:
n = LieAlgebra(QQ, cartan_type=['A',4], nilpotent=True)

In [ ]:
# Fix an enumeration of the positive roots s.t. ht(\beta_i)\leq ht(\beta_{i+1})



In [ ]:
so4 = lie_algebras.so(QQ, 5, representation='matrix'); so4
so4.gens()